In [ ]:
import warnings 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from nixtla import NixtlaClient
%matplotlib inline
warnings.filterwarnings("ignore")

API_KEY = 'YOUR-API-KEY'
nixtla_client = NixtlaClient(api_key = API_KEY)
nixtla_client.validate_api_key() # check your API key status



In [ ]:
train = pd.read_csv('../data/train.csv', parse_dates=['date'])
test = pd.read_csv('../data/test.csv', parse_dates=['date'])

In [ ]:
train.describe()

In [ ]:
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

##### Basic EDA
Let's understand the data a bit more

**Daily sales over time**

In [ ]:
daily_sale = train.groupby('date', as_index = False)['sales'].sum()
store_sale = train.groupby(['date','store'], as_index = False)['sales'].sum()
item_sale = train.groupby(['date','item'], as_index = False)['sales'].sum()


In [ ]:
# Plot using nixtla function 
nixtla_client.plot(daily_sale, time_col='date', target_col='sales')




In [ ]:
train.head(1)

#### Predict with TimeGPT
We will forecast daily sales for a full week 

In [ ]:
# add week and month to df
train['week'] = train['date'].dt.isocalendar().week
train['month'] = train['date'].dt.month
train_dummy = train_dummy[~train_dummy.index.duplicated()]
# one hot encode week and month
train_dummy = pd.get_dummies(train, columns=['week', 'month','store','item'], dtype=int)

In [ ]:
# Truncate data frame 
# forecast_df = train_dummy[train_dummy['delivery_week'] < "2016-10-16"]

# Let's use the last x weeks of actuals for the holdout set 
holdout_df = train_dummy[(train_dummy['date'] >= "2015-01-16") & (train_dummy['date'] <= "2017-04-29")]


In [ ]:
from nixtla.date_features import CountryHolidays

# create list with seasonal exogenous features
EXOGENOUS_FAETURES = [x for x in train_dummy.columns if ('week_' in x) | ('month_' in x) | ('item_' in x) | ('store_' in x) ]

timegpt_fcst_ex_vars_df = nixtla_client.forecast(
    df=train_dummy[['date', 'sales']+EXOGENOUS_FAETURES],
    time_col='date',
    target_col='sales',
    X_df=holdout_df[['date']+EXOGENOUS_FAETURES],
    h=17,
    level=[80],
    freq='W-MON',
    model='timegpt-1',
    add_history=True,
    )